# High Low (HiLo Activator)
O indicador HiLo é uma técnica para rastrear tendências, usando médias móveis, utilizado para identificar pontos de entrada e saída de operações.<br />
Os sinais de compra e venda deste indicador é baseado na alternância das séries de sinais.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import yfinance as yf
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
import plotly.subplots as pls
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Acoes = "CPLE3.SA"
Periodo_Inicio = datetime.datetime(2019,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
# Dados = pdr.get_data_yahoo(Acoes, start=Periodo_Inicio, end=Periodo_Termino)
Dados = yf.download(Acoes, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados

### Análise dos Preços
Candlestick da série de preços do ativo.

In [ ]:
SeriePrecos = plg.Figure(plg.Scatter(x=Dados.index, y=Dados.Close))
SeriePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(SeriePrecos, "Preços")

In [ ]:
CandlePrecos = plg.Figure(plg.Candlestick(x=Dados.index,
                                          open=Dados.Open,
                                          high=Dados.High,
                                          low=Dados.Low,
                                          close=Dados.Close))
CandlePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(CandlePrecos, "Preços")

### Cálculo do HiLo e séries de sinais
O HiLo é calculado a partir de duas médias móveis simples de curto prazo, normalmente de 8 dias, sendo uma a pártir do preço máximo e outra do preço mínimo.
- Alternando da linha de mínimo para máximo = Sinal para Compra.
- Alternando da linha de máximo para mínimo = Sinal para Venda.

In [ ]:
JanelaDeTempo = 8

In [ ]:
HiLo_Alta = Dados.High.rolling(window=JanelaDeTempo).mean()

In [ ]:
HiLo_Baixa = Dados.Low.rolling(window=JanelaDeTempo).mean()

In [ ]:
SerieHiLoAlta = pd.DataFrame(index=Dados.index)
SerieHiLoAlta["High"] = np.where(Dados.Close > HiLo_Alta, HiLo_Baixa, np.NaN)

In [ ]:
SerieHiLoBaixa = pd.DataFrame(index=Dados.index)
SerieHiLoBaixa["Low"] = np.where(Dados.Close < HiLo_Baixa, HiLo_Alta, np.NaN)

In [ ]:
SerieComMediaMovel = plg.Figure()
SerieComMediaMovel.add_trace(plg.Candlestick(x=Dados.index,
                                            open=Dados.Open,
                                            high=Dados.High,
                                            low=Dados.Low,
                                            close=Dados.Close,
                                            name="Preços",
                                            opacity=0.5))
SerieComMediaMovel.add_trace(plg.Scatter(mode="lines",
                                         x=SerieHiLoAlta.index,
                                         y=SerieHiLoAlta.High,
                                         name="HiLo Alta"))
SerieComMediaMovel.add_trace(plg.Scatter(mode="lines",
                                         x=SerieHiLoBaixa.index,
                                         y=SerieHiLoBaixa.Low,
                                         name="HiLo Baixa"))
Plot_Show(SerieComMediaMovel, "Preços & HiLo Activator")